# EDA - OpenAQ data

## Import library

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import requests
from pprint import pp
from tqdm.notebook import tqdm
import requests
import time
from zkyhaxpy import io_tools, pd_tools, gis_tools
from datetime import datetime, timedelta
import os

## Prepare GeoDataframe of OpenAQ data

In [ ]:
if not os.path.exists('../data/gdf_openaq_chiangmai_by_location.gpkg'):
    dir_data = r'../data'
    dir_openaq_chiangmai = os.path.join(dir_data, 'openaq', 'chiangmai_by_location')
    list_files = io_tools.get_list_files_re(dir_openaq_chiangmai)
    df_openaq = pd.concat([pd.read_csv(path_file) for path_file in list_files])
    
    
    df_openaq['datetime'] = df_openaq['date'].str.slice(9, 25)
    df_openaq['date'] = df_openaq['datetime'].str.slice(0, 10)
    df_openaq['year'] = df_openaq['date'].str.slice(0, 4)
    df_openaq['month'] = df_openaq['date'].str.slice(6, 8)
    df_openaq['year_month'] = df_openaq['date'].str.slice(0, 8)
    df_openaq['time'] = df_openaq['datetime'].str.slice(11, 17)
    df_openaq['lat'] = df_openaq['coordinates'].apply(lambda coord: ast.literal_eval(coord)['latitude'])
    df_openaq['lon'] = df_openaq['coordinates'].apply(lambda coord: ast.literal_eval(coord)['longitude'])
    df_openaq = df_openaq.reset_index()
    del(df_openaq['index'])
    del(df_openaq['coordinates'])

    df_openaq['measurement_id'] = df_openaq.index
    df_openaq.index.name = 'measurement_id'

    df_openaq['geometry'] = 'POINT (' + df_openaq['lon'].astype(str) + ' ' + df_openaq['lat'].astype(str) + ')'
    
    gdf_openaq = gis_tools.df_to_gdf(df_openaq, 'geometry')

    gdf_openaq = gdf_openaq.set_index('measurement_id')
    gdf_openaq.to_file('../data/gdf_openaq_chiangmai_by_location.gpkg')
    print('gdf_openaq has been saved.')
else:
    gdf_openaq = gpd.read_file('../data/gdf_openaq_chiangmai_by_location.gpkg')
    print('gdf_openaq has been loaded.')
if gdf_openaq.index.name is None:
    gdf_openaq = gdf_openaq.set_index('measurement_id')

In [ ]:
gdf_openaq.head()

In [ ]:
gdf_province = gpd.read_file(r'..\data\thailandWithName.json')
gdf_chiangmai = gdf_province[gdf_province['name'] == 'Chiang Mai']
gdf_chiangmai

In [ ]:
gdf_openaq.columns

In [ ]:
gdf_openaq_first_data = gdf_openaq.groupby('locationId').agg(first_data=('datetime', 'min'))
gdf_openaq_first_data

In [ ]:
gdf_openaq_location = gdf_openaq.drop(columns=['datetime', 'year', 'month', 'value', 'parameter', 'unit']).drop_duplicates()
gdf_openaq_location

# ['locationId', 'location', 'parameter', 'value', 'unit', 'country',
    #    'city', 'isMobile', 'isAnalysis', 'entity', 'sensorType', 'lat', 'long',
    #    'datetime', 'year', 'month', 'geometry']('')


In [ ]:
gdf_openaq_location_chiangmai = gdf_openaq_location[gdf_openaq_location.geometry.within(gdf_chiangmai.geometry.iloc[0])]
gdf_openaq_location_chiangmai

In [ ]:
gdf_openaq_location_chiangmai = gdf_openaq_location_chiangmai.drop_duplicates(subset=['locationId']).copy()
gdf_openaq_location_chiangmai

In [ ]:
gdf_openaq_location_chiangmai = gdf_openaq_location_chiangmai.merge(gdf_openaq_first_data, left_on='locationId', right_index=True)

In [ ]:
gdf_openaq_location_chiangmai.to_file('gdf_ope naq_location_chiangmai_v2.gpkg')